In [7]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import cufflinks as cf
import mytools.date as dt
import mytools.plot as mpl

cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Italian intensive care cases 

In [8]:
# loading the data
df = pd.read_csv('italy-intensive_care.csv')
a = list((dt.day_of_year_to_date(v)).strftime("%d/%m/%y") for v in df['day'])
df.index = a
fromDate = df['day'] >= 211
df.drop(columns=['day'])
# display the table


,intensive care
24/02/20,27
25/02/20,35
26/02/20,36
27/02/20,56
28/02/20,64
...,...
15/08/20,55
16/08/20,56
17/08/20,58
18/08/20,58


## Fitting intensive care data with exponential and logistic models

The exponential model is:

$$y = e^{k(x-x_0)} + y_0$$

where $k, x_0, y_0$ are the parameters to estimate.

For the logistic model:

$$y = \frac{c}{1+e^{-k(x-x_0)}} + y_0$$

where $c, k, x_0, y_0$ are the parameters to estimate.

In [9]:
condition = df['intensive care'] >= 0
df = df[condition & fromDate]

fig = mpl.iplot_analysis_plot(df.drop(columns=['day']), title='Intensive care cases', exp_fitting=False, sigm_fitting=False)
fig.show()

`ftol` termination condition is satisfied.
Function evaluations 41, initial cost 7.1819e-01, final cost 3.6476e-02, first-order optimality 1.31e-05.
        Sigmoid derivative model
        x0 = 233.5944640224776
        y0 = 40.23397982217318
        c = 399.34552853255815
        k = 0.22994518283205242
        max = 233.5944640224776, 239.90674408845226
        
